In [13]:
# This script relies on the results from Parcelization_Part1_MergeCounties.ipynb, which creates a gdb called Parcels_RPB.gdb
# Then, this script performs the parcelization calculation steps
# The results are clipped back to the non-buffered resource potential basemap boundary per county

In [14]:
import arcpy
import pandas as pd
import os

### Bring in result of Part 1

In [15]:
arcpy.env.workspace = "\\Parcels_RPB.gdb"
fc = arcpy.ListFeatureClasses()

In [16]:
base = "\\Parcels_RPB.gdb\\"

In [17]:
arcpy.env.workspace = os.getcwd()

### Bring in prepared data

In [18]:
solarRPB_3310 = "Parcelization_FullMethods.gdb\\TotalNewSolarRPB_IntCBuff" # same dataset as in Part 1
solarRPB_NB3310 = "Parcelization_FullMethods.gdb\\TotalNewSolarRPB_IntCounties" # solar resource map, intersected with the counties, not buffered. 
df = pd.read_excel('CountyList.xlsx') # was exported in Part 1

In [19]:
#define output coordinate system -- WKID 3310: create a spatial reference object for the output coordinate system
out_coordinate_system = arcpy.SpatialReference('NAD 1983 California (Teale) Albers (Meters)')

## Main Geoprocessing Steps of Parcelization Calculation

In [20]:
count = 0
for item in fc:
    cntyName = item.split('_')[0]
    startparcels = base+item
    cntyName2 = df[df['Unnamed: 0']==cntyName][0]
    print(startparcels)
    print(cntyName)
    print(cntyName2)
    
    tmpProj = "FinalParcelizationMap.gdb\\tmpProj"
    #project to WKID 3310
    arcpy.Project_management(startparcels, tmpProj, out_coordinate_system)
    
#     then perform parcelization steps
    nonNull = 'FinalParcelizationMap.gdb\\Test_Start'
    arcpy.analysis.Select(tmpProj, nonNull, 'PARCEL_APN IS NOT NULL')
    
    #rasterize
    outRas = r'FinalParcelizationMap.gdb\Test_Raster'
    arcpy.conversion.PolygonToRaster(nonNull, 
                                     "PARCEL_APN", 
                                     outRas, 
                                     "CELL_CENTER", 
                                     "NONE", 
                                     90, 
                                     "BUILD")
    
    #Nibble - remove null areas
    tmpNibble = r"FinalParcelizationMap.gdb\TmpNibble"
    fill = arcpy.sa.Con(arcpy.sa.IsNull(outRas),1,outRas)
    outRas2 = arcpy.sa.Nibble(fill,outRas,'DATA_ONLY')
    outRas2.save(tmpNibble)
    
    #focal Statistics
    FocalStats = r'FinalParcelizationMap.gdb\TestFocal'
    FocalStats = r'FocalStats_County.gdb\TestFocal_'+cntyName
    outRas3 = arcpy.ia.FocalStatistics(tmpNibble, 
                                       "Circle 804.672 MAP", 
                                       "VARIETY", 
                                       "DATA", 
                                       90)
    outRas3.save(FocalStats)
    
    #zonal Statistics
    zonalStats = r'FinalParcelizationMap.gdb\Test_ZonalStats'
    arcpy.sa.ZonalStatisticsAsTable(tmpNibble, 
                                    "Value", 
                                    FocalStats, 
                                    zonalStats, 
                                    "DATA", 
                                    "MEAN", 
                                    "CURRENT_SLICE", 
                                    [90], 
                                    "AUTO_DETECT", 
                                    "ARITHMETIC", 
                                    360)
    
    #bring the Mean value onto the original Raster attribute table with the Parcel APN
    arcpy.management.JoinField(zonalStats, 
                               "Value", 
                               outRas, 
                               "Value", 
                               "PARCEL_APN", 
                               "NOT_USE_FM", 
                               None)
    
    #join back to parcels polygon
    arcpy.management.JoinField(nonNull, 
                               "PARCEL_APN", 
                               zonalStats, 
                               "PARCEL_APN", 
                               "MEAN", 
                               "NOT_USE_FM", 
                               None)

    extractMeanNull = r'FinalParcelizationMap.gdb\Test_RemainingNull'
    arcpy.analysis.Select(nonNull, extractMeanNull, 'MEAN IS NULL')
    
    #create points of those null parcels
    centroidPoints = r'FinalParcelizationMap.gdb\Test_NullPoints'
    arcpy.management.FeatureToPoint(extractMeanNull, centroidPoints, 'CENTROID')
    
    #get value from raster (nibble raster) onto point
    ValueRaster = r"FinalParcelizationMap.gdb\Extract_NibbleValue_Test"
    arcpy.sa.ExtractValuesToPoints(centroidPoints,
                                   tmpNibble, 
                                   ValueRaster, 
                                   "NONE", 
                                   "VALUE_ONLY")
    
    #bring the Mean value onto the original Raster attribute table with the PArcel APN
    arcpy.management.JoinField(ValueRaster, 
                               "RASTERVALU", 
                               zonalStats, 
                               "Value", 
                               "MEAN", 
                               "NOT_USE_FM", 
                               None)
    
    # bring data back together
    arcpy.management.JoinField(nonNull, 
                               "PARCEL_APN",
                               ValueRaster, 
                               "PARCEL_APN", 
                               "MEAN_1", 
                               "NOT_USE_FM", 
                               None)
 
    arcpy.AddField_management(nonNull, "Mean_Combined", "FLOAT")
    fields = ['Mean_Combined','MEAN','MEAN_1']
    with arcpy.da.UpdateCursor(nonNull,fields) as cursor:
        for row in cursor:
            if row[1] == None:
                row[0] = row[2]
            else:
                row[0] = row[1]
                
            cursor.updateRow(row)
       
     #select SolarRPB_CountiesINTERSECT       
    cntyRPB = 'FinalParcelizationMap.gdb\\cntyRPB_tmp'
    whereClause = "NAME = "+"'"+cntyName2[count]+ "'"
    thisTest = arcpy.analysis.Select(solarRPB_NB3310, cntyRPB, whereClause)
    
    finalOut = 'Parcels_NewRPB.gdb\\'+cntyName+"_MeanParcelization"
    arcpy.analysis.Clip(nonNull,cntyRPB,finalOut)

    count+=1
    





\Parcels_RPB.gdb\ALAMEDA_RPBParcels
ALAMEDA
0    Alameda County
Name: 0, dtype: object
\Parcels_RPB.gdb\ALPINE_RPBParcels
ALPINE
1    Alpine County
Name: 0, dtype: object
\Parcels_RPB.gdb\AMADOR_RPBParcels
AMADOR
2    Amador County
Name: 0, dtype: object
\Parcels_RPB.gdb\BUTTE_RPBParcels
BUTTE
3    Butte County
Name: 0, dtype: object
\Parcels_RPB.gdb\CALAVERAS_RPBParcels
CALAVERAS
4    Calaveras County
Name: 0, dtype: object
\Parcels_RPB.gdb\COLUSA_RPBParcels
COLUSA
5    Colusa County
Name: 0, dtype: object
\Parcels_RPB.gdb\CONTRACOSTA_RPBParcels
CONTRACOSTA
6    Contra Costa County
Name: 0, dtype: object
\Parcels_RPB.gdb\DELNORTE_RPBParcels
DELNORTE
7    Del Norte County
Name: 0, dtype: object
\Parcels_RPB.gdb\ELDORADO_RPBParcels
ELDORADO
8    El Dorado County
Name: 0, dtype: object
\Parcels_RPB.gdb\FRESNO_RPBParcels
FRESNO
9    Fresno County
Name: 0, dtype: object
\Parcels_RPB.gdb\GLENN_RPBParcels
GLENN
10    Glenn County
Name: 0, dtype: object
\Parcels_RPB.gdb\HUMBOLDT_RPBParcels
HU

In [1]:
# #check that feature classes were written
arcpy.env.workspace = "Parcels_NewRPB.gdb"
fc = arcpy.ListFeatureClasses()

['ALAMEDA_MeanParcelization', 'ALPINE_MeanParcelization', 'AMADOR_MeanParcelization', 'BUTTE_MeanParcelization', 'CALAVERAS_MeanParcelization', 'COLUSA_MeanParcelization', 'CONTRACOSTA_MeanParcelization', 'DELNORTE_MeanParcelization', 'ELDORADO_MeanParcelization', 'FRESNO_MeanParcelization', 'GLENN_MeanParcelization', 'HUMBOLDT_MeanParcelization', 'IMPERIAL_MeanParcelization', 'INYO_MeanParcelization', 'KERN_MeanParcelization', 'KINGS_MeanParcelization', 'LAKE_MeanParcelization', 'LASSEN_MeanParcelization', 'LOSANGELES_MeanParcelization', 'MADERA_MeanParcelization', 'MARIN_MeanParcelization', 'MARIPOSA_MeanParcelization', 'MENDOCINO_MeanParcelization', 'MERCED_MeanParcelization', 'MODOC_MeanParcelization', 'MONO_MeanParcelization', 'MONTEREY_MeanParcelization', 'NAPA_MeanParcelization', 'NEVADA_MeanParcelization', 'ORANGE_MeanParcelization', 'PLACER_MeanParcelization', 'PLUMAS_MeanParcelization', 'RIVERSIDE_MeanParcelization', 'SACRAMENTO_MeanParcelization', 'SANBENITO_MeanParcelizatio

In [2]:
#Union counties together to create statewide parcelization map
fieldmappings = arcpy.FieldMappings()
fieldmap = arcpy.FieldMap()
outMerge = "FinalParcelizationMap.gdb\\MergeAll_Parcelization_NewRPB"
allFCs = []
for item in fc:
    totalName = "Parcels_NewRPB.gdb\\"+item 
    #Merge all parcels together (all tmpOutClip) 
    addSourceInfo = "ADD_SOURCE_INFO"
    
    fieldmappings.addTable(totalName)
    allFCs.append(totalName)
    

arcpy.management.Merge(allFCs, outMerge, fieldmappings, addSourceInfo)
    
    
    

<Result 'C:\\Users\\shossain\\OneDrive - California Energy Commission\\SpatialAnalysis\\LandUse_EnergyResource\\Parcelization\\LSA_Method\\FinalParcelizationMap.gdb\\MergeAll_Parcelization_NewRPB'>